In [2]:
import pandas as pd
import numpy as np
import math, random
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import pyprind
import os

In [4]:
df = pd.read_csv('movie_data.csv')
df.head(3)

,review,sentiment
0,In Canadian director Kari Skogland's film adap...,1
1,"I agree with the previous comment, what a disa...",0
2,"If you are looking for King Kong, you mispelle...",1


In [5]:
from sklearn.feature_extraction.text import CountVectorizer

[[0 1 1 1 0 1 0]
 [0 1 0 0 1 1 1]
 [1 2 1 1 1 2 1]]


In [6]:
from sklearn.feature_extraction.text import TfidfTransformer

In [7]:
def tfidf(bag):
    tf = []
    nd = float(bag.shape[0])
    unnorm = bag.multiply((np.log((1+nd)/(1.0+bag.sum(axis=0)))+1.0)) #multiply bag(tf) by (idf+1)
    for v in unnorm:
        tf.append(v/np.linalg.norm(v))
    return np.array(tf)

In [8]:
from sklearn.preprocessing import normalize

In [9]:
import re

In [10]:
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

In [12]:
df['review'] = df['review'].apply(preprocessor)

In [13]:
def tokenizer(text):
    return text.split()

In [14]:
from nltk.stemm.porter import PorterStemmer

ImportError: No module named stemm.porter

In [ ]:
porter = PorterStemmer()

In [ ]:
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [ ]:
import nltk

In [91]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Steve\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [15]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
print stop

[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u

In [16]:
xtrain = df.loc[:25000, 'review'].values
ytrain = df.loc[:25000, 'sentiment'].values
xtest = df.loc[25000:, 'review'].values
ytest = df.loc[25000:, 'sentiment'].values

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)

TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [19]:
lr_tfidf = Pipeline([('vect', TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None, stop_words=None, 
                            tokenizer=tokenizer, ngram_range=(1,1))), 
                     ('clf', LogisticRegression(random_state=0, C=10.0, penalty='l2'))])

In [20]:
lr_tfidf.fit(xtrain, ytrain)

Pipeline(steps=[('vect', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=Tru...ovr',
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0))])

In [21]:
from sklearn.metrics import accuracy_score

In [22]:
print "Training accuracy:", accuracy_score(ytrain, lr_tfidf.predict(xtrain))

Training accuracy: 0.988680452782


In [23]:
print "Test accuracy:", accuracy_score(ytest, lr_tfidf.predict(xtest))

Test accuracy: 0.89696


In [24]:
def stream_docs(path):
    with open(path, 'r') as csv:
        next(csv) #skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [25]:
next(stream_docs(path='movie_data.csv'))

('"In Canadian director Kari Skogland\'s film adaptation of the Margaret Laurence novel The Stone Angel Ellen Burstyn is Hagar Shipley, a proud and cantankerous woman approaching her nineties who wishes to remain independent until the very end, stubbornly refusing to be placed in a nursing home by her well-meaning son Marvin. Filmed in Manitoba, Canada and set in the fictional town of Manawaka, The Stone Angel is a straightforward and conventional interpretation of the book that has been required reading in Canadian high school English classes for almost half a century.<br /><br />The title of the film comes from the stone statue erected on Hagar\'s mother\'s grave which serves as a metaphor for Hagar\'s inability to express emotion during her tumultuous lifetime. Burstyn brings vulnerability and humor to the role but is a bit too likable to fully realize the ego-driven, self-defeating character who managed to alienate her wealthy father, her well-meaning but alcoholic husband, and bot

In [26]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [27]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

In [28]:
vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

clf = SGDClassifier(loss='log', random_state=1, n_iter=1)
doc_stream = stream_docs(path='movie_data.csv')

In [29]:
pbar = pyprind.ProgBar(45)
classes = np.array([0,1])
for _ in range(45):
    xtrain, ytrain = get_minibatch(doc_stream, size=1000) #train with 1000 documents at a time
    if not xtrain:
        break
    xtrain = vect.transform(xtrain)
    clf.partial_fit(xtrain, ytrain, classes=classes)
    pbar.update()

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:15


In [30]:
xtest, ytest = get_minibatch(doc_stream, size=5000)
xtest = vect.transform(xtest)
print "Accuracy:", clf.score(xtest, ytest)

Accuracy: 0.8212


In [31]:
clf = clf.partial_fit(xtest, ytest)

In [32]:
import pickle

In [34]:
dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)

pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol=2)   
pickle.dump(clf, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=2)

In [35]:
%%writefile movieclassifier/vectorizer.py #writes contents of cell to directory and file location
from sklearn.feature_extraction.text import HashingVectorizer
import re
import os
import pickle

cur_dir = os.path.dirname(__file__)
stop = pickle.load(open(
                os.path.join(cur_dir, 
                'pkl_objects', 
                'stopwords.pkl'), 'rb'))

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
                   + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)

Writing movieclassifier/vectorizer.py
